# TextVibCLIP Vibration Encoder 단독 성능 테스트

UOS 데이터셋에서 Vibration Encoder가 진동 신호를 제대로 분류할 수 있는지 검증


In [1]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.manifold import TSNE
from collections import Counter
import logging
import seaborn as sns

# 프로젝트 루트 추가
sys.path.append('/data/home/kyj2024/TextVibCLIP')

from src.vibration_encoder import VibrationEncoder
from src.data_loader import BearingDataset
from torch.utils.data import DataLoader
from src.data_loader import create_collate_fn
from configs.model_config import MODEL_CONFIG

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🔧 Vibration Encoder 단독 성능 테스트 시작")


/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Vibration Encoder 단독 성능 테스트 시작


/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

## 1. 데이터 준비


In [2]:
# UOS 데이터셋 로드
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

# Train 데이터
train_dataset = BearingDataset(
    data_dir='data_scenario1',
    dataset_type='uos',
    domain_value=600,
    subset='train'
)

# Test 데이터
test_dataset = BearingDataset(
    data_dir='data_scenario1',
    dataset_type='uos',
    domain_value=600,
    subset='test'
)

print(f"Train 데이터: {len(train_dataset)}개 샘플")
print(f"Test 데이터: {len(test_dataset)}개 샘플")

# 샘플 확인
sample = train_dataset[0]
print(f"\n샘플 구조:")
print(f"  라벨: {sample['labels']}")
print(f"  진동 신호 shape: {sample['vibration'].shape}")
print(f"  진동 신호 범위: [{sample['vibration'].min():.4f}, {sample['vibration'].max():.4f}]")


INFO:src.data_loader:UOS 데이터 라벨 분포: {'B_6204': 1, 'H_6204': 4, 'IR_6204': 1, 'OR_6204': 1}
INFO:src.data_loader:최소 샘플 수: 1
INFO:src.data_loader:UOS Domain-Incremental 윈도우 레벨 분할:
INFO:src.data_loader:  모든 subset에 모든 7개 파일 포함
INFO:src.data_loader:  각 파일 내에서 윈도우 분할: Train 60%, Val 20%, Test 20%
INFO:src.data_loader:  Deep Groove Ball 7-클래스 분포: {'B': 1, 'H': 1, 'IR': 1, 'OR': 1, 'L': 1, 'M': 1, 'U': 1}
INFO:src.data_loader:  클래스 수: 7개 (균형 확인)
INFO:src.data_loader:  ✅ 완벽한 클래스 균형 달성!
INFO:src.data_loader:UOS train 분할 결과:
INFO:src.data_loader:  Train: 7개 파일, Val: 7개 파일, Test: 7개 파일


디바이스: cuda


INFO:src.data_loader:BearingDataset 초기화 완료 (UOS): 7개 파일, 1249개 윈도우/파일, 총 8743개 샘플, Domain: 600, Subset: train
INFO:src.data_loader:UOS 데이터 라벨 분포: {'B_6204': 1, 'H_6204': 4, 'IR_6204': 1, 'OR_6204': 1}
INFO:src.data_loader:최소 샘플 수: 1
INFO:src.data_loader:UOS Domain-Incremental 윈도우 레벨 분할:
INFO:src.data_loader:  모든 subset에 모든 7개 파일 포함
INFO:src.data_loader:  각 파일 내에서 윈도우 분할: Train 60%, Val 20%, Test 20%
INFO:src.data_loader:  Deep Groove Ball 7-클래스 분포: {'B': 1, 'H': 1, 'IR': 1, 'OR': 1, 'L': 1, 'M': 1, 'U': 1}
INFO:src.data_loader:  클래스 수: 7개 (균형 확인)
INFO:src.data_loader:  ✅ 완벽한 클래스 균형 달성!
INFO:src.data_loader:UOS test 분할 결과:
INFO:src.data_loader:  Train: 7개 파일, Val: 7개 파일, Test: 7개 파일
INFO:src.data_loader:BearingDataset 초기화 완료 (UOS): 7개 파일, 1249개 윈도우/파일, 총 8743개 샘플, Domain: 600, Subset: test
INFO:src.data_loader:인덱스 매핑 생성 완료: 8743개 (파일 7개 × 윈도우 1249개)


Train 데이터: 8743개 샘플
Test 데이터: 8743개 샘플

샘플 구조:
  라벨: tensor([6, 0])
  진동 신호 shape: torch.Size([2048])
  진동 신호 범위: [-1.5622, 1.7830]


## 2. Vibration Encoder 생성 및 진동 임베딩


In [3]:
# Vibration Encoder 생성 (올바른 파라미터 사용)
vibration_config = MODEL_CONFIG['vibration_encoder']
embedding_dim = MODEL_CONFIG['embedding_dim']

vibration_encoder = VibrationEncoder(
    input_length=vibration_config['input_length'],
    embedding_dim=embedding_dim
)

vibration_encoder.to(device)
vibration_encoder.eval()

# 파라미터 수 계산
total_params = sum(p.numel() for p in vibration_encoder.parameters())
trainable_params = sum(p.numel() for p in vibration_encoder.parameters() if p.requires_grad)

print(f"Vibration Encoder 정보:")
print(f"  입력 길이: {vibration_config['input_length']}")
print(f"  출력 차원: {embedding_dim}")
print(f"  총 파라미터: {total_params:,}개")
print(f"  학습 파라미터: {trainable_params:,}개")
print(f"  아키텍처: {vibration_config['kernel_sizes']} kernels, {vibration_config['channels']} channels")

# 테스트 입력으로 출력 차원 확인
with torch.no_grad():
    test_input = torch.randn(1, vibration_config['input_length']).to(device)
    test_output = vibration_encoder(test_input)
    print(f"  실제 출력 shape: {test_output.shape}")
    print(f"  정규화 후 범위: [{test_output.min().item():.4f}, {test_output.max().item():.4f}]")


INFO:src.vibration_encoder:1D-CNN VibrationEncoder 초기화: input_length=2048, embedding_dim=256
INFO:src.vibration_encoder:   OPTIMIZED: 커널 크기: [16, 32, 64, 32] - 4-layer 베어링 최적화
INFO:src.vibration_encoder:   OPTIMIZED: 채널 수: [64, 128, 256, 512] - 자연스러운 64→512 증가
INFO:src.vibration_encoder:   총 파라미터: 6,985,288


Vibration Encoder 정보:
  입력 길이: 2048
  출력 차원: 256
  총 파라미터: 6,985,288개
  학습 파라미터: 6,985,288개
  아키텍처: [16, 32, 64, 32] kernels, [64, 128, 256, 512] channels
  실제 출력 shape: torch.Size([1, 256])
  정규화 후 범위: [-0.0825, 0.0676]


## 3. 진동 데이터 샘플링 및 임베딩 생성


In [4]:
# 클래스별 진동 데이터 수집 (각 클래스당 100개씩)
class_vibrations = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: []}  # 7개 클래스
class_labels = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
samples_per_class = 100
class_counts = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

print("클래스별 진동 데이터 수집 중...")
for i in range(len(train_dataset)):
    if all(count >= samples_per_class for count in class_counts.values()):
        break
        
    sample = train_dataset[i]
    label = sample['labels'][0].item()  # 주 분류
    vibration = sample['vibration']
    
    if class_counts[label] < samples_per_class:
        class_vibrations[label].append(vibration)
        class_labels[label].append(label)
        class_counts[label] += 1

print(f"\n수집된 클래스별 진동 데이터 수: {class_counts}")

# 모든 진동 데이터를 하나의 텐서로 결합
all_vibrations = []
all_labels = []

for class_id, vibrations in class_vibrations.items():
    all_vibrations.extend(vibrations)
    all_labels.extend(class_labels[class_id])

# 텐서로 변환
vibration_tensor = torch.stack(all_vibrations)  # (700, 2048)
labels_tensor = torch.tensor(all_labels)

print(f"전체 진동 데이터: {vibration_tensor.shape}")
print(f"클래스 분포: {Counter(all_labels)}")
print(f"진동 신호 통계: mean={vibration_tensor.mean():.4f}, std={vibration_tensor.std():.4f}")


클래스별 진동 데이터 수집 중...

수집된 클래스별 진동 데이터 수: {0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100}
전체 진동 데이터: torch.Size([700, 2048])
클래스 분포: Counter({0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100})
진동 신호 통계: mean=-0.0004, std=1.0121


## 4. 진동 임베딩 생성 및 분석


In [5]:
# 진동 임베딩 생성
print("진동 임베딩 생성 중...")
vibration_embeddings_list = []
batch_size = 32

with torch.no_grad():
    for i in range(0, len(vibration_tensor), batch_size):
        batch_vibrations = vibration_tensor[i:i+batch_size].to(device)
        batch_embeddings = vibration_encoder(batch_vibrations)
        batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
        vibration_embeddings_list.append(batch_embeddings.cpu())

vibration_embeddings = torch.cat(vibration_embeddings_list, dim=0)

print(f"Vibration embeddings shape: {vibration_embeddings.shape}")
print(f"임베딩 통계: mean={vibration_embeddings.mean().item():.4f}, std={vibration_embeddings.std().item():.4f}")
print(f"임베딩 범위: min={vibration_embeddings.min().item():.4f}, max={vibration_embeddings.max().item():.4f}")

unique_classes = torch.unique(labels_tensor)
print(f"고유 클래스: {unique_classes.tolist()}")


진동 임베딩 생성 중...
Vibration embeddings shape: torch.Size([700, 256])
임베딩 통계: mean=-0.0034, std=0.0624
임베딩 범위: min=-0.1840, max=0.1650
고유 클래스: [0, 1, 2, 3, 4, 5, 6]


## 5. 진동 신호 분류 성능 테스트 (핵심)


In [6]:
# 클래스별 prototype 계산
class_prototypes = []
prototype_labels = []

print("클래스별 진동 prototype 생성:")
for cls in unique_classes:
    cls_mask = (labels_tensor == cls)
    cls_embeddings = vibration_embeddings[cls_mask]
    cls_prototype = cls_embeddings.mean(dim=0, keepdim=True)
    
    class_prototypes.append(cls_prototype)
    prototype_labels.append(cls)
    
    # 클래스 내 유사도 분석
    if len(cls_embeddings) > 1:
        intra_sim = torch.matmul(cls_embeddings, cls_embeddings.t())
        # 대각선 제외한 평균 (자기 자신 제외)
        mask = ~torch.eye(len(cls_embeddings), dtype=torch.bool)
        intra_mean = intra_sim[mask].mean().item()
        intra_std = intra_sim[mask].std().item()
        print(f"  클래스 {cls.item()}: {len(cls_embeddings)}개, 내부 유사도 {intra_mean:.4f}±{intra_std:.4f}")

# Prototype 결합
prototype_matrix = torch.cat(class_prototypes, dim=0)  # (7, 256)
prototype_labels = torch.stack(prototype_labels)  # (7,)

print(f"\nPrototype matrix shape: {prototype_matrix.shape}")
print(f"Prototype labels: {prototype_labels.tolist()}")

# 🎯 핵심: 진동 신호 분류 성능 테스트
similarities = torch.matmul(vibration_embeddings, prototype_matrix.t())  # (700, 7)
predicted_indices = torch.argmax(similarities, dim=1)  # (700,)

# 디바이스 일치 확인
if predicted_indices.device != prototype_labels.device:
    predicted_indices = predicted_indices.to(prototype_labels.device)

predicted_classes = prototype_labels[predicted_indices]  # (700,)

# 정확도 계산
accuracy = (predicted_classes == labels_tensor).float().mean().item()
print(f"\n🔧 Vibration Encoder 분류 정확도: {accuracy:.4f} ({accuracy*100:.1f}%)")

# 이론적 랜덤 베이스라인
random_baseline = 1.0 / len(unique_classes)
print(f"이론적 랜덤 베이스라인: {random_baseline:.4f} ({random_baseline*100:.1f}%)")
print(f"랜덤 대비 향상: {accuracy/random_baseline:.2f}배")

# 클래스별 정확도
print("\n클래스별 정확도:")
class_names = ['H', 'B', 'IR', 'OR', 'L', 'U', 'M']
class_accuracies = []
for cls in unique_classes:
    cls_mask = (labels_tensor == cls)
    cls_correct = (predicted_classes[cls_mask] == labels_tensor[cls_mask]).sum().item()
    cls_total = cls_mask.sum().item()
    cls_acc = cls_correct / cls_total if cls_total > 0 else 0
    class_accuracies.append(cls_acc)
    print(f"  클래스 {cls.item()} ({class_names[cls.item()]}): {cls_correct}/{cls_total} = {cls_acc:.4f} ({cls_acc*100:.1f}%)")

# 혼동 행렬
cm = confusion_matrix(labels_tensor.cpu(), predicted_classes.cpu())
print(f"\n혼동 행렬:")
print(cm)


클래스별 진동 prototype 생성:
  클래스 0: 100개, 내부 유사도 0.9985±0.0004
  클래스 1: 100개, 내부 유사도 0.9977±0.0007
  클래스 2: 100개, 내부 유사도 0.9909±0.0031
  클래스 3: 100개, 내부 유사도 0.9968±0.0008
  클래스 4: 100개, 내부 유사도 0.9936±0.0025
  클래스 5: 100개, 내부 유사도 0.9943±0.0022
  클래스 6: 100개, 내부 유사도 0.9975±0.0010

Prototype matrix shape: torch.Size([7, 256])
Prototype labels: [0, 1, 2, 3, 4, 5, 6]

🔧 Vibration Encoder 분류 정확도: 0.8857 (88.6%)
이론적 랜덤 베이스라인: 0.1429 (14.3%)
랜덤 대비 향상: 6.20배

클래스별 정확도:
  클래스 0 (H): 100/100 = 1.0000 (100.0%)
  클래스 1 (B): 98/100 = 0.9800 (98.0%)
  클래스 2 (IR): 66/100 = 0.6600 (66.0%)
  클래스 3 (OR): 100/100 = 1.0000 (100.0%)
  클래스 4 (L): 91/100 = 0.9100 (91.0%)
  클래스 5 (U): 95/100 = 0.9500 (95.0%)
  클래스 6 (M): 70/100 = 0.7000 (70.0%)

혼동 행렬:
[[100   0   0   0   0   0   0]
 [  2  98   0   0   0   0   0]
 [  0   0  66   5  29   0   0]
 [  0   0   0 100   0   0   0]
 [  0   0   1   8  91   0   0]
 [  0   5   0   0   0  95   0]
 [ 30   0   0   0   0   0  70]]


## 6. 결론 및 진단


In [7]:
print("\n" + "="*60)
print("🔧 Vibration Encoder 성능 진단 결과")
print("="*60)

# 클래스 구분도 계산
inter_class_sim = torch.matmul(prototype_matrix, prototype_matrix.t())
diag_sim = torch.diag(inter_class_sim).mean().item()
off_diag_mask = ~torch.eye(len(unique_classes), dtype=torch.bool)
off_diag_sim = inter_class_sim[off_diag_mask].mean().item()
separation_score = diag_sim - off_diag_sim

print(f"1. 분류 정확도: {accuracy*100:.1f}% (랜덤: {random_baseline*100:.1f}%)")
print(f"2. 클래스 구분도: {separation_score:.4f}")
print(f"3. 임베딩 품질: std={vibration_embeddings.std().item():.4f}")
print(f"4. 최고 클래스 정확도: {max(class_accuracies)*100:.1f}%")
print(f"5. 최저 클래스 정확도: {min(class_accuracies)*100:.1f}%")

# 종합 진단
print(f"\n🔍 종합 진단:")
if accuracy > 0.8:
    diagnosis = "우수"
    next_step = "Text-Vibration 정렬 문제 집중 분석"
    color = "✅"
elif accuracy > 0.6:
    diagnosis = "양호"
    next_step = "Vibration Encoder 아키텍처 개선"
    color = "⚠️"
elif accuracy > 0.4:
    diagnosis = "보통"
    next_step = "데이터 전처리 및 정규화 개선"
    color = "⚠️"
else:
    diagnosis = "불량"
    next_step = "Vibration Encoder 아키텍처 재설계"
    color = "❌"

print(f"{color} Vibration Encoder 성능: {diagnosis} ({accuracy*100:.1f}%)")
print(f"📋 다음 단계: {next_step}")

# 세부 권장사항
print(f"\n🎯 세부 권장사항:")
if accuracy < 0.5:
    print("- 1D-CNN 아키텍처 재설계 (더 깊은 네트워크)")
    print("- 데이터 정규화 방법 변경")
    print("- 윈도우 크기 조정 (2048 → 4096)")
elif accuracy < 0.7:
    print("- Dropout 비율 조정")
    print("- 커널 크기 최적화")
    print("- 배치 정규화 추가")
else:
    print("- Vibration Encoder는 정상 작동")
    print("- Text-Vibration 정렬 문제에 집중")

# Text Encoder vs Vibration Encoder 비교
text_encoder_performance = 0.83  # Text Encoder 성능
textvib_performance = 0.1443  # TextVibCLIP 전체 성능

print(f"\n📈 성능 비교:")
print(f"  Text Encoder 단독: {text_encoder_performance*100:.1f}%")
print(f"  Vibration Encoder 단독: {accuracy*100:.1f}%")
print(f"  TextVibCLIP 전체: {textvib_performance*100:.1f}%")

if accuracy > 0.7 and text_encoder_performance > 0.7:
    print("🚨 두 Encoder 모두 정상 - 정렬/통합 과정에 심각한 문제")
elif accuracy < 0.5:
    print("🔍 Vibration Encoder 개선 필요")
else:
    print("⚠️ 부분적 문제 - 추가 분석 필요")

print(f"\n🔧 아키텍처 정보:")
print(f"  커널 크기: {vibration_config['kernel_sizes']}")
print(f"  채널 수: {vibration_config['channels']}")
print(f"  드롭아웃: {vibration_config['dropout']}")
print(f"  입력→출력: {vibration_config['input_length']} → {embedding_dim}")
print(f"  총 파라미터: {total_params:,}개")



🔧 Vibration Encoder 성능 진단 결과
1. 분류 정확도: 88.6% (랜덤: 14.3%)
2. 클래스 구분도: 0.0073
3. 임베딩 품질: std=0.0624
4. 최고 클래스 정확도: 100.0%
5. 최저 클래스 정확도: 66.0%

🔍 종합 진단:
✅ Vibration Encoder 성능: 우수 (88.6%)
📋 다음 단계: Text-Vibration 정렬 문제 집중 분석

🎯 세부 권장사항:
- Vibration Encoder는 정상 작동
- Text-Vibration 정렬 문제에 집중

📈 성능 비교:
  Text Encoder 단독: 83.0%
  Vibration Encoder 단독: 88.6%
  TextVibCLIP 전체: 14.4%
🚨 두 Encoder 모두 정상 - 정렬/통합 과정에 심각한 문제

🔧 아키텍처 정보:
  커널 크기: [16, 32, 64, 32]
  채널 수: [64, 128, 256, 512]
  드롭아웃: 0.1
  입력→출력: 2048 → 256
  총 파라미터: 6,985,288개
